In [11]:
import numpy as np
import tensorflow as tf 
import gym
import tensorflow_probability as tfp
import tensorflow.keras.losses as kls

In [16]:
from tensoraerospace.envs.unity_env import get_plane_env, unity_discrete_env

In [180]:
class critic(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(128,activation='relu')
        self.v = tf.keras.layers.Dense(1, activation = None)

    def call(self, input_data):
        x = self.d1(input_data)
        v = self.v(x)
        return v
    

class actor(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(128,activation='relu')
        self.a = tf.keras.layers.Dense(3 ** 7,activation='softmax')
        self.r = tf.keras.layers.Dense(1, activation='relu')

    def call(self, input_data, return_reward=False):
        x = self.d1(input_data)
        a = self.a(x)
        if return_reward:
            r = self.r(x)
            return a, r
        return a



In [187]:
class agent():
    def __init__(self, gamma = 0.99):
        self.gamma = gamma
        # self.a_opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
        # self.c_opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
        self.a_opt = tf.keras.optimizers.Adam(learning_rate=7e-3)
        self.c_opt = tf.keras.optimizers.Adam(learning_rate=7e-3)
        self.actor = actor()
        self.critic = critic()
        self.clip_pram = 0.2

          
    def act(self,state):
        prob = self.actor(np.array([state]))
        prob = prob.numpy()
        dist = tfp.distributions.Categorical(probs=prob, dtype=tf.float32)
        action = dist.sample()
        if int(action.numpy()[0]) >= 3 ** 7:
            return int(action.numpy()[0]) - 1
        return int(action.numpy()[0])
  


    def actor_loss(self, probs, actions, adv, old_probs, closs):
        
        probability = probs      
        entropy = tf.reduce_mean(tf.math.negative(tf.math.multiply(probability,tf.math.log(probability))))
        #print(probability)
        #print(entropy)
        sur1 = []
        sur2 = []
        
        for pb, t, op,a  in zip(probability, adv, old_probs, actions):
            t =  tf.constant(t)
            #print(a)
            #op =  tf.constant(op)
            #print(f"t{t}")
            #ratio = tf.math.exp(tf.math.log(pb + 1e-10) - tf.math.log(op + 1e-10))
            ratio = tf.math.divide(pb[a],op[a])
            #print(f"ratio{ratio}")
            s1 = tf.math.multiply(ratio,t)
            #print(f"s1{s1}")
            s2 =  tf.math.multiply(tf.clip_by_value(ratio, 1.0 - self.clip_pram, 1.0 + self.clip_pram),t)
            #print(f"s2{s2}")
            sur1.append(s1)
            sur2.append(s2)

        sr1 = tf.stack(sur1)
        sr2 = tf.stack(sur2)
        
        #closs = tf.reduce_mean(tf.math.square(td))
        loss = tf.math.negative(tf.reduce_mean(tf.math.minimum(sr1, sr2)) - closs + 0.001 * entropy)
        #print(loss)
        return loss
    
    def auxillary_task(self, r, rewards):
        loss = tf.reduce_mean(tf.math.square(r - rewards))
        return loss

    def learn(self, states, actions,  adv , old_probs, discnt_rewards, rewards):
        discnt_rewards = tf.reshape(discnt_rewards, (len(discnt_rewards),))
        adv = tf.reshape(adv, (len(adv),))

        old_p = old_probs

        old_p = tf.reshape(old_p, (len(old_p),3 ** 7))
        with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            p, r = self.actor(states, return_reward=True, training=True)
            v =  self.critic(states,training=True)
            v = tf.reshape(v, (len(v),))
            td = tf.math.subtract(discnt_rewards, v)
            c_loss = 0.5 * kls.mean_squared_error(discnt_rewards, v)
            a_loss = self.actor_loss(p, actions, adv, old_probs, c_loss) + self.auxillary_task(r, rewards)
            
        grads1 = tape1.gradient(a_loss, self.actor.trainable_variables)
        grads2 = tape2.gradient(c_loss, self.critic.trainable_variables)
        self.a_opt.apply_gradients(zip(grads1, self.actor.trainable_variables))
        self.c_opt.apply_gradients(zip(grads2, self.critic.trainable_variables))
        return a_loss, c_loss

In [182]:
def test_reward(env):
    #print("yes")
    total_reward = 0
    state = env.reset()
    done = False
    for i in range(10):
        action = np.argmax(agentoo7.actor(np.array([state])).numpy())
        next_state, reward, done, _ = env.step(action)
        state = next_state
        total_reward += reward
        done = True

    return total_reward

In [189]:
env= unity_discrete_env(env_path="C:\\Users\\DexFrost89\\3_step\\builds\\My project.exe")
def preprocess1(states, actions, rewards, done, values, gamma):
    g = 0
    lmbda = 0.95
    returns = []
    for i in reversed(range(len(rewards))):
        delta = rewards[i] + gamma * values[i + 1] * done[i] - values[i]
        g = delta + gamma * lmbda * dones[i] * g
        returns.append(g + values[i])

    returns.reverse()
    adv = np.array(returns, dtype=np.float32) - values[:-1]
    adv = (adv - np.mean(adv)) / (np.std(adv) + 1e-10)
    states = np.array(states, dtype=np.float32)
    actions = np.array(actions, dtype=np.int32)
    returns = np.array(returns, dtype=np.float32)
    rewards = np.array(rewards, dtype=np.float32)
    return states, actions, returns, adv, rewards    


tf.random.set_seed(336699)
agentoo7 = agent()
steps = 10
ep_reward = []
total_avgr = []
target = False 
best_reward = 0
avg_rewards_list = []


for s in range(steps):
    if target == True:
        break
  
    done = False
    state = env.reset()
    all_aloss = []
    all_closs = []
    rewards = []
    states = []
    actions = []
    probs = []
    dones = []
    values = []
    print("new episod")

    for e in range(10):
   
        action = agentoo7.act(state)
        value = agentoo7.critic(np.array([state])).numpy()
        next_state, reward, done, _ = env.step(action)
        dones.append(1-done)
        rewards.append(reward)
        states.append(state)
        #actions.append(tf.one_hot(action, 2, dtype=tf.int32).numpy().tolist())
        actions.append(action)
        prob = agentoo7.actor(np.array([state]))
        probs.append(prob[0])
        values.append(value[0][0])
        state = next_state
        if done:
      env.reset()
  
  value = agentoo7.critic(np.array([state])).numpy()
  values.append(value[0][0])
  np.reshape(probs, (len(probs),3 ** 7))
  probs = np.stack(probs, axis=0)

  states, actions,returns, adv, rewards  = preprocess1(states, actions, rewards, dones, values, 1)

  for epocs in range(1):
      al,cl = agentoo7.learn(states, actions, adv, probs, returns, rewards)
      # print(f"al{al}") 
      # print(f"cl{cl}")   

  avg_reward = np.mean([test_reward(env) for _ in range(5)])
  print(f"total test reward is {avg_reward}")
  avg_rewards_list.append(avg_reward)
  if avg_reward > best_reward:
        print('best reward=' + str(avg_reward))
        agentoo7.actor.save('model_actor_{}_{}'.format(s, avg_reward), save_format="tf")
        agentoo7.critic.save('model_critic_{}_{}'.format(s, avg_reward), save_format="tf")
        best_reward = avg_reward
  if best_reward == 200:
        target = True
  env.reset()

env.close()
    
  

[INFO] Connected to Unity environment with package version 2.2.1-exp.1 and communication version 1.5.0
[INFO] Connected new brain: My Behavior?team=0
[WARNING] uint8_visual was set to true, but visual observations are not in use. This setting will not have any effect.
[WARNING] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.
new episod
total test reward is -10.0
new episod
total test reward is -10.0
new episod
total test reward is -10.0
new episod
total test reward is -10.0
new episod
total test reward is -10.0
new episod
total test reward is -10.0
new episod
total test reward is -10.0
new episod
total test reward is -10.0
new episod
total test reward is -10.0
new episod
total test reward is -10.0
